In [ ]:
CREATE OR REPLACE TABLE `yhcr-prd-bradfor-bia-core.CB_2172.cb_2172_efi2_hypotension_union` AS (

with hypotension_numeric as (
  (with a as (
SELECT 
person_id, 
ctv3text,
cast(numericvalue as numeric) as numericvalue_1,
 idappointment, 
 dateevent, 
 extract(date from dateevent) as date_record
,case when ctv3text like '%iastolic%' then 'Diastolic'
when ctv3text like '%ystolic%' then 'Systolic'
when snomedcode in('163033001','163034007','163035008') then 'Systolic'else NULL end as bp_reading
 FROM `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode` where snomedcode in(SELECT SNOMEDCT_CONCEPTID FROM `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Hypotension / syncope' AND Otherinstructions IS NOT NULL) and 
 person_id in (select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)
order by dateeventrecorded), 

b as (
  select a.*,
  cast(c.first_episodestartdate as datetime) first_episodestartdate
  from a
  left join `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` c
  on a.person_id = c.person_id
  #i cleared bo_readings that were not specifically systolic or diastolic and zero bp values, vast majority of bp_Reading that were not specifically systolic or diastolic had zero numericvalues so happy to exclude
  ),
#select for the minimum systlic and diastolic value on a given date 
d as(
  select 
  person_id, date_record,bp_reading,
  min(numericvalue_1) as min_bp
  from b
  where dateevent > datetime_sub(first_episodestartdate, INTERVAL 60 MONTH) and dateevent < first_episodestartdate
  group by person_id, date_record, bp_reading
  order by person_id, date_record),
#create a new varibale of the preceding and following systolic and distolic value 
 min_bp_reading as (select
  person_id,
  bp_reading,
  min_bp,
  lag(min_bp) over (partition by person_id, bp_reading order by date_record) as prev_reading,
  lag(date_record)over(partition by person_id, bp_reading order by date_record) as prev_reading_date,
  lead(min_bp) over (partition by person_id, bp_reading order by date_record) as next_reading,
  lead(date_record) over (partition by person_id, bp_reading order by date_record) as next_reading_date,
  date_record,
  from d
  order by person_id, date_record),
  #filter for those where the preceding and following readings are within 12 months. 
   hypotension_variable as 
(select
*,
case when bp_reading in ('Systolic') and min_bp < 90 and prev_reading < 90 and next_reading < 90 then 'yes'
when bp_reading in('Diastolic') and min_bp < 60 and prev_reading < 60 and next_reading < 60 then 'yes'
else NULL end as hypotension
from min_bp_reading
where date_diff(next_reading_date,prev_reading_date,month)<12)

select distinct person_id
from hypotension_variable
where hypotension is not null)
),

#then code for hypontnsion by diagnosis code

hypotension_diagnosis as (
 with a as (
  SELECT 
  person_id,
  dateevent,
  row_number() over(partition by person_id order by dateevent) code_sequence
  FROM  `yhcr-prd-bradfor-bia-core.CB_FDM_PrimaryCare.tbl_srcode`
  WHERE snomedcode in(SELECT SNOMEDCT_CONCEPTID FROM `yhcr-prd-bradfor-bia-core.ref_Lookups.lkp_EFI2_Codelist` where deficit = 'Hypotension / syncope' AND Otherinstructions IS NULL) AND person_id in(select person_id from `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1`)),

  snomed as (
  SELECT 
  m.person_id,
  m.first_episodestartdate,
  a.dateevent as hypotension_date,
  FROM `yhcr-prd-bradfor-bia-core.CB_2172.care_home_cohort_v1` m
  LEFT JOIN a ON
  a.person_id= m.person_id AND a.dateevent < cast(m.first_episodestartdate as datetime) AND a.dateevent > date_sub(m.date_of_admission, interval 5 year) AND code_sequence = 1),

cohort as (
select
person_id,
case when hypotension_date is not null then 'yes'
when hypotension_date is null then 'no'
end as hypotension_diagnosis
from snomed)

select
*
from cohort
where hypotension_diagnosis = 'yes'),

hypotension_union as (
SELECT person_id FROM hypotension_numeric

union distinct

Select person_id from hypotension_diagnosis)

select
person_id,
 case when person_id IS NOT NULL then 'Yes' end as hypotension_diagnosis
 from hypotension_union)

